In [7]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-132734")
#exp = Experiment(workspace=ws, name="udacity-project-hyper")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

#run = exp.start_logging()

Workspace name: quick-starts-ws-132734
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-132734


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster-2"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing Comput Target")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

### YOUR CODE HERE ###

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, loguniform
import os

# Specify parameter sampler
ps1 = RandomParameterSampling(
     {"--C": loguniform(-3,2), "--max_iter": choice(1,5,10,20,30,40,50,80,100,200,400,800,1000)}
)

### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
arguments = {"--C": 1.0, "--max_iter": 100}
current_dir = os.getcwd()
est = SKLearn(source_directory=current_dir, 
              entry_script="train.ipynb",
              compute_target = compute_target,
              script_params = arguments)

### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps1,
    primary_metric_name="Accuracy",
    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=2
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run_hyper = exp.submit(config=hyperdrive_config)
### YOUR CODE HERE ###

In [ ]:
run_hyper.wait_for_completion(show_output=True)

In [ ]:
import joblib
best_run = run_hyper.get_best_run_by_primary_metric()

In [ ]:
print(best_run.get_file_names())

In [ ]:
model = best_run.register_model(workspace=ws, model_name="best-hyper-model", model_path="outputs/model.pkl")

In [29]:
# Revisit the best model run
from azureml.core.run import get_run
run = get_run(exp, "HD_ff6daa53-4190-4c58-b79e-96b27f799cb4")
children = run.get_children()
childs = [*children]
best_score = 0
best_child = None
for child in childs:
    print(child.get_metrics())
    metrics = child.get_metrics()
    if metrics and 'Accuracy' in metrics.keys() and metrics['Accuracy'] > best_score:
        best_score = child.get_metrics()['Accuracy']
        best_child = child

{'Regularization Strength:': 0.07304859840588435, 'Max iterations:': 50, 'Accuracy': 0.9072837632776934}
{'Regularization Strength:': 0.8783856468955435, 'Max iterations:': 30, 'Accuracy': 0.9072837632776934}
{'Regularization Strength:': 0.07815523037726442, 'Max iterations:': 200, 'Accuracy': 0.9072837632776934}
{'Regularization Strength:': 0.19954366367073706, 'Max iterations:': 1000, 'Accuracy': 0.9072837632776934}
{'Regularization Strength:': 1.1174328674115732, 'Max iterations:': 1, 'Accuracy': 0.887556904400607}
{'Regularization Strength:': 0.059588646057997595, 'Max iterations:': 20, 'Accuracy': 0.9072837632776934}
{'Regularization Strength:': 0.07931390798235143, 'Max iterations:': 40, 'Accuracy': 0.9072837632776934}
{'Regularization Strength:': 0.3036966103888117, 'Max iterations:': 200, 'Accuracy': 0.9072837632776934}
{'Regularization Strength:': 5.53221402077548, 'Max iterations:': 1, 'Accuracy': 0.887556904400607}
{'Regularization Strength:': 1.956289473217793, 'Max iterati

In [ ]:
exp_auto = Experiment(workspace=ws, name="udacity-project-automl")
run_auto = exp_auto.start_logging()

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", header=True, validate=True)

In [ ]:
# Somehow the loading of the local module "train.py" doesn't work instead the clean_data method is directly initilized in the 
# udacity-project Jupyter Notebook
import pandas as pd

def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    return x_df, y_df

In [ ]:
# from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['y'] = y

In [ ]:
from collections import Counter 
Counter(x['y'])

In [ ]:
# Somehow the registered Dataset cannot be used for AutoML 
from azureml.core import Dataset
dataset_training = Dataset.from_pandas_dataframe(x)
dataset_training = dataset_training.register(workspace=ws, name="auto-ml-training-data", description="example exercise data")

In [ ]:
# To be able to use a AutoML model on a Dataset it is necessary to save it e.g. in Azureml Blob Storage and then load it from 
# there
import os
from azureml.core import Dataset, Datastore, Workspace
os.makedirs("data", exist_ok=True)

dataset_path = "data/train_set.csv"
x.to_csv(dataset_path)
datastore = ws.get_default_datastore()
x.to_csv(dataset_path)
datastore.upload(src_dir="data", target_path="data")

In [ ]:
train_data = Dataset.Tabular.from_delimited_files(path = [(datastore, ("data/train_set.csv"))])

In [ ]:
train_data.to_pandas_dataframe()

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=5)

In [ ]:
# Submit your automl run
run_auto = exp_auto.submit(automl_config, show_output=True)

In [ ]:
run_auto.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.
dir(run_auto)
### YOUR CODE HERE ###

In [ ]:
best_auto_run = run_auto.get_best_child()

In [ ]:
best_auto_run.register_model(model_name="best_automl_model", model_path="outputs/model.pkl")

In [ ]:
dir(best_auto_run)